In [1]:
import mne

In [2]:
m2 = mne.io.read_raw_eeglab("/om/data/public/language-eeg/frank2015/EEG02.set", preload=True)
m1 = mne.io.read_raw_eeglab("/om/data/public/language-eeg/frank2015/EEG01.set", preload=True)

# NB EEG data recorded during sentence boundaries is marked as NaN in the raw data. this is awkward. maybe best to split data at NaN boundaries, then rebuild with `concatenate_raw` (so boundaries don't get messed up during epoching)

In [3]:
mne.preprocessing.annotate_break(m1)

Ignoring annotations with descriptions starting with: bad, edge

Detected 3 break periods of >= 15.0 sec duration:
    561.8 – 618.0 sec [56.3 sec]
    1202.5 – 1261.4 sec [59.0 sec]
    1882.8 – 1915.8 sec [33.0 sec]
In total, 5.8% of the data (148.2 sec) have been marked as a break.



<Annotations | 3 segments: BAD_break (3)>

In [7]:
m1_mat = m1.get_data()

In [21]:
import numpy as np

# Get indices into raw EEG data where stimulus presentation begins
# (i.e. index i s.t. data[i] is not NaN and data[i - 1] is NaN)
presentation_begins = np.where(~np.isnan(m1_mat[0, 1:]) & np.isnan(m1_mat[0, :-1]))[0]
presentation_ends = np.where(~np.isnan(m1_mat[0, :-1]) & np.isnan(m1_mat[0, 1:]))[0]
presentation_spans = np.array(list(zip(presentation_begins, presentation_ends)))
presentation_spans

array([[  2430,   4106],
       [  5313,   7780],
       [  8827,  11202],
       [ 12394,  13888],
       [ 14702,  17020],
       [ 17863,  20042],
       [ 21141,  22453],
       [ 23831,  25864],
       [ 27077,  28917],
       [ 29928,  30876],
       [ 31483,  32791],
       [ 33499,  35863],
       [ 36463,  37615],
       [ 39271,  40237],
       [ 41126,  42231],
       [ 43288,  44789],
       [ 45539,  47689],
       [ 48335,  49612],
       [ 50918,  52405],
       [ 54093,  55202],
       [ 56598,  58088],
       [ 58898,  61081],
       [ 62926,  65119],
       [ 65851,  68159],
       [ 70139,  71994],
       [ 74335,  76528],
       [ 77559,  78939],
       [ 79871,  81208],
       [ 82522,  83819],
       [ 85615,  87101],
       [ 88308,  89791],
       [ 90527,  92870],
       [ 94009,  96362],
       [ 97890,  99919],
       [100997, 102801],
       [103887, 105934],
       [107483, 108609],
       [109769, 110725],
       [111585, 113589],
       [114179, 115298],


In [29]:
# Sanity check: no overlapping spans.
assert not (presentation_spans[:-1, 1] > presentation_spans[1:, 0]).any()

In [22]:
m1.info

<Info | 7 non-empty values
 bads: []
 ch_names: VEOG, HEOG, 50, 36, 49, 37, 48, 38, 47, 39, 46, 40, 45, 41, 44, ...
 chs: 34 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 34
 projs: []
 sfreq: 250.0 Hz
>

In [ ]:
m1_events, m1_event_id = mne.events_from_annotations(m1)
m1_epochs = mne.Epochs(m1, events=m1_events, event_id=m1_event_id, preload=True)
m1_epochs.average().plot()